In [1]:
import pandas as pd
import numpy as np
import os
import json

from math import floor, ceil, log, exp
from glob import glob
from tqdm import tqdm
from scipy.signal import decimate
from scipy.stats import binom
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class Alarm:
    def __init__(self, start, timer_duration, detection_interval, max_time):
        self.start = start
        self.end = start + timer_duration
        self.warning_start = start + detection_interval
        self.timer_duration = timer_duration
        self.detection_interval = detection_interval
        self.max_time = max_time
        self.seizures = []

    def duration(self):
        return self.end - self.start

    def contains(self, time):
        return self.start<=time<=self.end

    def warns_seizure(self, time):
        return self.warning_start<=time<=self.end

    def add_seizures(self, seizures):
        for seizure in seizures:
            if self.warns_seizure(seizure):
                self.seizures.append(seizure)

    def extend_from(self, time):
        self.end = time + self.timer_duration
        if self.end > self.max_time:
            self.end = self.max_time

In [3]:
patients = ['MSEL_00172', 'MSEL_00501', 'MSEL_01097', 'MSEL_01110-ICU', 'MSEL_01575', 'MSEL_01808', 'MSEL_01838', 'MSEL_01842', 'MSEL_01844']
patient_id = patients[0]

In [4]:
def predict_segment(segment):
    return np.random.randint(2)

In [5]:
def predict_patient(patient_id, prediction_function, probability_threshold=0.5):
    labels = pd.read_csv(f"data\data\{patient_id}\{patient_id}_labels.csv")
    n_segments = int((labels["duration"][0]-labels["duration"][0]%30000)/30000)

    predictions = pd.DataFrame(data={"prediction_probability":np.nan, "time":np.nan}, index=pd.RangeIndex(start=0, stop=n_segments, step=1))
    for segment_id in range(n_segments):
        try:
            segment = pd.read_parquet(f"data/segments/test/{patient_id}/{patient_id}_test_segment_{segment_id}.parquet")
        except FileNotFoundError:
            continue
        predictions.loc[segment_id, "time"] = segment.index[-1] + 250
        predictions.loc[segment_id, "prediction_probability"] = prediction_function(segment)
    predictions = predictions.loc[:predictions["prediction_probability"].last_valid_index()]
    predictions["prediction"] = predictions["prediction_probability"] > probability_threshold
    predictions = predictions.dropna(axis=0)
    return predictions

In [6]:
def get_seizure_times(patient_id):
    labels = pd.read_csv(f"data\data\{patient_id}\{patient_id}_labels.csv")
    start_time = labels["startTime"][0]
    seizure_times = labels["labels.startTime"] - start_time
    return seizure_times

In [7]:
def get_triggers(predictions, integration_windows, thresholds):
    moving_averages = pd.DataFrame(data={"time":predictions["time"]}, index=predictions.index)
    triggers = pd.DataFrame(data={"time":predictions["time"]}, index=predictions.index)
    for window_size in integration_windows:
        moving_averages[window_size] = predictions["prediction"].rolling(int(window_size/30000)).mean()
        for threshold in thresholds:
            triggers[f"{window_size}_{threshold}"] = moving_averages[window_size] > threshold
    return triggers

In [8]:
def get_alarms(triggers, timer_duration, detection_interval, recording_duration):
    alarms = {}
    hyperparameters = triggers.columns[triggers.columns!="time"]
    for hyperparameter in hyperparameters:
        hyperparameter_trigger_times = triggers.loc[triggers[hyperparameter], "time"]
        alarms[hyperparameter] = []
        current_alarm = Alarm(hyperparameter_trigger_times.iloc[0], timer_duration, detection_interval, recording_duration)
        for trigger_time in hyperparameter_trigger_times:
            if current_alarm.contains(trigger_time):
                current_alarm.extend_from(trigger_time)
            else:
                alarms[hyperparameter].append(current_alarm)
                current_alarm = Alarm(trigger_time, timer_duration, detection_interval, recording_duration)
        alarms[hyperparameter].append(current_alarm)
    return alarms

In [9]:
def p_value(n_correct, n_seizures, chance_sensitivity):
    if n_correct/n_seizures > chance_sensitivity:
        k_f = floor(2*n_seizures*chance_sensitivity-n_correct)
        return 1-binom.cdf(n_correct-1, n_seizures, chance_sensitivity)+binom.cdf(k_f, n_seizures, chance_sensitivity)
    elif  n_correct/n_seizures < chance_sensitivity:
        k_c = ceil(2*n_seizures*chance_sensitivity-n_correct)
        return 1-binom.cdf(k_c-1, n_seizures, chance_sensitivity)+binom.cdf(n_correct, n_seizures, chance_sensitivity)
    else:
        raise Exception("Hey Bailey")

In [10]:
def get_metrics(alarms, seizure_times, recording_duration):
    detection_interval = alarms[0].detection_interval
    timer_duration = alarms[0].timer_duration
    n_correct = sum([len(alarm.seizures) for alarm in alarms])
    n_seizures = len(seizure_times)
    sensitivity = n_correct/n_seizures
    time_in_warning = sum([alarm.duration() for alarm in alarms])/recording_duration

    # tau_w = timer_duration/recording_duration
    # tau_w0 = detection_interval/recording_duration
    rate_parameter = (-1/timer_duration)*log(1-time_in_warning)
    chance_sensitivity = 1 - exp((-rate_parameter*timer_duration)+(1 - exp(-rate_parameter*detection_interval)))
    improvement_over_chance = sensitivity - chance_sensitivity
    p = p_value(n_correct, n_seizures, chance_sensitivity)
    return {"S": sensitivity, "TiW": time_in_warning, "IoC": improvement_over_chance, "p": p}

In [11]:
integration_windows = [600000, 300000]
thresholds = [0.4, 0.5, 0.6]
timer_duration = 450000
detection_interval = 60000

In [12]:
predictions = predict_patient(patient_id, predict_segment)

In [15]:
def evaluate(patient_id, predictions):
    recording_duration = predictions["time"].max()
    seizure_times = get_seizure_times(patient_id)
    seizure_times = seizure_times[seizure_times<recording_duration]
    triggers = get_triggers(predictions, integration_windows, thresholds)
    print(triggers)
    alarms = get_alarms(triggers, timer_duration, detection_interval, recording_duration)

    metrics = {}
    for hyperparameter, hyperparameter_alarms in alarms.items():
        for alarm in hyperparameter_alarms:
            alarm.add_seizures(seizure_times)
        metrics[hyperparameter] = get_metrics(hyperparameter_alarms, seizure_times, recording_duration)
    return metrics

In [16]:
evaluate(patient_id, predictions)

Empty DataFrame
Columns: [time, 600000_0.4, 600000_0.5, 600000_0.6, 300000_0.4, 300000_0.5, 300000_0.6]
Index: []


IndexError: single positional indexer is out-of-bounds

In [ ]:
def p_value(n_correct, n_seizures, chance_sensitivity)
    

[600000.0]

In [ ]:
p_value(49, 100, 0.5)

0.9204107626128206

In [ ]:
labels = pd.read_csv(f"data\data\{patient_id}\{patient_id}_labels.csv")
start_time = labels["startTime"][0]
seizure_times = labels["labels.startTime"] - start_time

with open("labels.json") as f:
    label_notes = json.load(f)

seizure_indexes = np.floor(seizure_times/30000)
seizure_indexes = seizure_indexes.loc[np.array([label_notes[note] for note in labels["labels.note"]])==1]

In [ ]:
seizure_times

0      13104000
1      27272000
2      67902000
3      78890000
4      89870000
5      92195000
6      93707000
7     163948000
8     166401000
9     248170000
10    274770000
11    279230000
Name: labels.startTime, dtype: int64

In [ ]:
p_value(8, 10, 0.5)

0.10937500000000001